In [5]:
import pandas as pd
import numpy as np
from dwave.system import DWaveSampler, EmbeddingComposite, LeapHybridSampler
from dimod import SimulatedAnnealingSampler

# Load the QUBO matrix and genes data (compute this on the MATLAB code)
df_genes = pd.read_csv('genes.csv', header=None)
df = pd.read_csv('qubo_matrix.csv', header=None)

# Convert QUBO matrix to numpy array
Qmat = np.asarray(df)
print("QUBO matrix shape:", Qmat.shape)

# Convert QUBO matrix to dictionary format
def qubo_matrix_to_dict(Q):
    qubo = {}
    for i in range(Q.shape[0]):
        for j in range(Q.shape[1]):
            if Q[i, j] != 0:
                qubo[(i, j)] = Q[i, j]
    return qubo

qubo = qubo_matrix_to_dict(Qmat)

# Quantum (Hybrid) Solver
print("Quantum (Hybrid) Solver Results:")

# Using Leap Hybrid Sampler
sampler_q = LeapHybridSampler()
sampleset_q = sampler_q.sample_qubo(qubo)

# Retrieve and analyze quantum results
#print("Best solution: ", sampleset_q.first.sample)
print("Energy: ", sampleset_q.first.energy)
print("Occurrences: ", sampleset_q.first.num_occurrences)

# Convert total runtime from microseconds (10^-6) to seconds
runtime_all = sampleset_q.info.get('run_time', 0)  # Provide a default value in case the key is missing
runtime_all_seconds = runtime_all / 1_000_000

# Convert QPU access time from microseconds (10^-6) to seconds
annealing_time = sampleset_q.info.get('qpu_access_time', 0)  # Provide a default value in case the key is missing
annealing_time_seconds = annealing_time / 1_000_000

print(f"Quantum annealing time: {annealing_time_seconds} seconds")
print(f"D-Wave hybrid solver time: {runtime_all_seconds} seconds")

# Store selected features from quantum solution
sample_dict_q = sampleset_q.first.sample
values_q = list(sample_dict_q.values())
# Get dataframe with gene's information
df_genes['feature_selected'] = values_q
df_genes = df_genes.rename(columns={0: 'Gene'})

filt_df_q = df_genes[df_genes['feature_selected'] > 0].copy()

# Convert feature_selected to numpy array (in this case it's just a vector of ones)
qa_sol = np.asarray(filt_df_q['feature_selected'])

# Convert filt_df_q to use row numbers as indices
row_indices = filt_df_q.index.to_numpy()

# Extract corresponding rows from Qmat using row indices
Qfilt = Qmat[:, row_indices]

# Multiply selected rows by qa_sol (it is a vector of ones so the result is the sum of selected rows)
result = Qfilt @ qa_sol
result = result[row_indices]

# Include gene score to selected features
filt_df_q.loc[:, 'gene_score'] = list(result)
filt_df_q_sort = filt_df_q.sort_values(by='gene_score', ascending=True)

filt_df_q_sort.to_csv('filt_df_QA.csv', index=True)

# Classical Solver
# print("\nClassical Solver Results:")

# Initialize the classical sampler
# sampler_c = SimulatedAnnealingSampler()

# Sample the QUBO
# sampleset_c = sampler_c.sample_qubo(qubo, num_reads=100)

# Retrieve and analyze classical results
# print("Best solution: ", sampleset_c.first.sample)
# print("Energy: ", sampleset_c.first.energy)
# print("Occurrences: ", sampleset_c.first.num_occurrences)

# Store selected features from classical solution
# sample_dict_c = sampleset_c.first.sample
# values_c = list(sample_dict_c.values())
# df_genes['feature_selected'] = values_c
# filt_df_c = df_genes[df_genes['feature_selected'] > 0]
# filt_df_c.to_csv('filt_df_SA.csv', index=True)

# filt_df_q, filt_df_c

QUBO matrix shape: (5000, 5000)
Quantum (Hybrid) Solver Results:
Energy:  -2.359038467439544
Occurrences:  1
Quantum annealing time: 0.079944 seconds
D-Wave hybrid solver time: 14.598281 seconds


In [8]:
# Embedding problem for this large matrix, it is too complex for the available resources.
# import dimod
# import dwave.system
## Create a sampler using D-Wave's EmbeddingComposite
# sampler_q0 = dwave.system.EmbeddingComposite(dwave.system.DWaveSampler())
## Sample the QUBO
#sampleset_q0 = sampler_q0.sample_qubo(qubo, num_reads=100)
## Print sample results for QUBO (optional)
#print(sampleset_q0)
